In [59]:
from IPython.display import JSON

import pandas as pd

import altair as alt
# Uncomment/run this line to enable Altair in JupyterLab/nteract:
alt.enable_mime_rendering()

import ipywidgets as widgets
from ipywidgets import fixed, interact

from openfisca_core import decompositions, periods
from openfisca_france import FranceTaxBenefitSystem
from openfisca_matplotlib import graphs

In [60]:
tbf = FranceTaxBenefitSystem()

## Display "Revenu disponible" interactively

In [61]:
def calculate_revenu_disponible(salaire_de_base, period):
    scenario_params = {
        "period": period,
        "parent1": {
            "age": 30,
            "salaire_de_base": salaire_de_base, # Annual basis
        },
    }
    scenario = tbs.new_scenario().init_single_entity(**scenario_params)
    simulation = scenario.new_simulation()
    revenu_disponible = simulation.calculate("revenu_disponible", period)
    return revenu_disponible

Try with an input text:

In [62]:
interact(
    calculate_revenu_disponible,
    salaire_de_base=widgets.BoundedFloatText(min=0, max=150000, step=100, value=15000, description="Salaire de base"),
    period=fixed(periods.period("2015")),
)
None  # Hide strange output

interactive(children=(BoundedFloatText(value=15000.0, description=u'Salaire de base', max=150000.0, step=100.0), Output()), _dom_classes=('widget-interact',))

Note: press Enter after you type a new value to recompute.

Try with a slider:

In [53]:
interact(
    calculate_revenu_disponible,
    salaire_de_base=widgets.FloatSlider(min=0, max=150000, step=100, value=15000, continuous_update=False),
    period=fixed(periods.period("2015")),
)
None  # Hide strange output

interactive(children=(FloatSlider(value=15000.0, continuous_update=False, description=u'salaire_de_base', max=150000.0, step=100.0), Output()), _dom_classes=('widget-interact',))

Note: you can edit the number at the right of the slider. Press Enter after you type a new value to recompute.

This is quite slow to recompute and redraw if we don't use `continuous_update=False`.
This is mainly because we create a new simulation each time, in `calculate_revenu_disponible`. I did not find how to update a variable of an existing simulation, I'm not sure it is possible with OpenFisca.

## Display waterfall

In [63]:
def display_waterfall(salaire_de_base, period):
    scenario_params = {
        "period": period,
        "parent1": {
            "age": 30,
            "salaire_de_base": salaire_de_base, # Annual basis
        },
    }
    scenario = tbs.new_scenario().init_single_entity(**scenario_params)
    simulation = scenario.new_simulation()
    graphs.draw_waterfall(simulation)

In [64]:
interact(
    draw_waterfall,
    salaire_de_base=widgets.FloatSlider(min=0, max=150000, step=100, value=15000, continuous_update=False),
)
None  # Hide strange output

interactive(children=(FloatSlider(value=15000.0, continuous_update=False, description=u'salaire_de_base', max=150000.0, step=100.0), Output()), _dom_classes=('widget-interact',))

Test with Vega

In [52]:
decomposition_json = decompositions.get_decomposition_json(simulation.tax_benefit_system)

In [59]:
JSON(decomposition_json)

<IPython.core.display.JSON object>

In [69]:
def iter_decomposition_leaves(node):
    children = node.get("children")
    if children:
        for child_node in children:
            for leave in iter_decomposition_leaves(child_node):
                yield leave
    else:
        yield node

In [71]:
decomposition_leaves = list(iter_decomposition_leaves(decomposition_json))

In [73]:
scenario_period = periods.period("2015")

for decomposition_leave in decomposition_leaves:
    variable_name = decomposition_leave["code"]
    simulation.calculate_add(variable_name, scenario_period)

In [ ]:
waterfall_df = pd.DataFrame()

## Vary "Salaire" and create a "bareme"